In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql.window import Window

In [2]:
list=  [['a',1,6],
        ['a',2,8],
        ['a',3,9],
        ['a',10,11],
        ['a',11,12],
        ['a',12,11]]
df=spark.createDataFrame(list,['key','number','value'])
df.show()

+---+------+-----+
key|number|value|
+---+------+-----+
 a| 1| 6|
 a| 2| 8|
 a| 3| 9|
 a| 10| 11|
 a| 11| 12|
 a| 12| 11|
+---+------+-----+

In [3]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import when
w=Window().partitionBy("key").orderBy("number")
w2=Window().partitionBy("key").orderBy("number").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df.withColumn("number2", F.lag("number").over(w)).withColumn("diff", F.when((F.col("number2").isNotNull()) & ((F.col("number")-F.col("number2")) > 1), (F.col("number")-F.col("number2"))).otherwise(F.lit(0)))\
.withColumn("diff2", F.lead("diff").over(w)).withColumn("diff2", F.when(F.col("diff2").isNull(), F.lit(0)).otherwise(F.col("diff2"))).withColumn("diff2", F.when(F.col("diff2")!=0, F.col("diff2")-1).otherwise(F.col("diff2"))).withColumn("max", F.max("number").over(w2))\
.withColumn("diff2", F.when((F.col("number")==F.col("max")) & (F.col("number")<F.lit(12)), F.lit(12)-F.col("number")).otherwise(F.col("diff2")))\
.withColumn("number2", F.when(F.col("diff2")!=0,F.expr("""sequence(number,number+diff2,1)""")).otherwise(F.expr("""sequence(number,number+diff2,0)""")))\
.drop("diff","diff2","max")\
.withColumn("number2", F.explode("number2")).drop("number")\
.select("key", F.col("number2").alias("number"), "value")\
.show()


+---+------+-----+
key|number|value|
+---+------+-----+
 a| 1| 6|
 a| 2| 8|
 a| 3| 9|
 a| 4| 9|
 a| 5| 9|
 a| 6| 9|
 a| 7| 9|
 a| 8| 9|
 a| 9| 9|
 a| 10| 11|
 a| 11| 12|
 a| 12| 11|
+---+------+-----+

In [5]:
.withColumn("number2", F.when(F.col("diff2")!=0,F.expr("""sequence(number,diff2,1)""")).otherwise(F.expr("""sequence(number,diff2,1)""")))\

In [6]:
key    number    value
a      1         6
a      2         10
a      3         10
a      4         10
a      5         20
a      6         20
a      7         20
a      8         20
a      9         25
a      10        25
a      11        25
a      12        25